In [1]:
import pandas as pd
pd.DataFrame.iteritems = pd.DataFrame.items

import numpy as np
import pickle
import logging
import multiprocessing as mp
from tqdm.contrib.concurrent import process_map

import os, sys

/data/home/umang/miniconda3/envs/mtm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sim_name = "l3harris" # "l3harris" or "novateur"
out_dir = f"/data/home/umang/Trajectory_project/anomaly_traj_data/stitch_mount/datassd1_8tb/umang/p2t4_exp_outs/sp_dfs/{sim_name}__train/"

# df.head()

In [3]:
os.getcwd()

'/data/home/umang/Trajectory_project/GPS-MTM'

In [4]:
poi_csv_data = f"/data/home/umang/Trajectory_project/anomaly_traj_data/stitch_mount/datassd3_8tb/p1t3_iarpa_data/phase1_trial3/{sim_name}/trial/poi.csv"

try:
    df_poi = pd.read_csv(poi_csv_data)
except FileNotFoundError:
    print(f"File not found: {poi_csv_data}")
    df_poi = None

lat_lons_count_dict = {}
lat_lons_types_dict = {}
for i in range(len(df_poi)):
    df_poi_lat = df_poi.iloc[i]["lat"]
    df_poi_lon = df_poi.iloc[i]["lon"]
    df_poi_category = df_poi.iloc[i]["poi_category"]

    if (df_poi_lat, df_poi_lon) not in lat_lons_count_dict:
        lat_lons_count_dict[(df_poi_lat, df_poi_lon)] = 1
        lat_lons_types_dict[(df_poi_lat, df_poi_lon)] = set([df_poi_category])
    else:
        lat_lons_count_dict[(df_poi_lat, df_poi_lon)] += 1
        lat_lons_types_dict[(df_poi_lat, df_poi_lon)].add(df_poi_category)
        
poi_types_list = [(item, len(item.split("__"))) for item in set(["__".join(sorted(list(item))) for item in lat_lons_types_dict.values()])]

poi_types_odered = sorted([(str(item[1]).zfill(3)+ "__" +item[0]) for item in poi_types_list])
poi_types_odered_tuples = [(i, poi_types_odered[i]) for i in range(len(poi_types_odered))]

binary_rep_of_pois = dict()
for id, poi_type in poi_types_odered_tuples:
    poi_type_list = poi_type.split("__")[1:]
    binary_rep_of_pois[poi_type] = [0] * (len(poi_types_odered)+2)
    binary_rep_of_pois[poi_type][id] = 1

poi_embeddings_dict = dict()

for lat_lon in lat_lons_types_dict:
    poi_string = "__".join(sorted(list(lat_lons_types_dict[lat_lon])))
    poi_types_count = len(poi_string.split("__"))
    
    poi_string_full = str(poi_types_count).zfill(3) + "__" + poi_string
    poi_embedding = binary_rep_of_pois[poi_string_full]
    poi_embeddings_dict[lat_lon] = poi_embedding
    

File not found: /data/home/umang/Trajectory_project/anomaly_traj_data/stitch_mount/datassd3_8tb/p1t3_iarpa_data/phase1_trial3/l3harris/trial/poi.csv


TypeError: object of type 'NoneType' has no len()

In [4]:
def get_nighttime_stops(df):
    # Convert timestamps to datetime
    df["stay_start_timestamp"] = pd.to_datetime(df["stay_start_timestamp"])
    df["stay_end_timestamp"] = pd.to_datetime(df["stay_end_timestamp"])
    
    # Define the nighttime period (8 PM to 6 AM)
    night_start = 20  # 8 PM
    night_end = 6  # 6 AM

    def calculate_intersection(row):
        total_intersection = 0
        current = row["stay_start_timestamp"]
        end = row["stay_end_timestamp"]
        
        while current < end:
            if (current.hour >= night_start) or (current.hour < night_end):
                next_hour = current.replace(minute=0, second=0) + pd.Timedelta(hours=1)
                next_time = min(next_hour, end)
                total_intersection += (next_time - current).total_seconds()
            current += pd.Timedelta(hours=1)
        
        return total_intersection

    df["intersection_time"] = df.apply(calculate_intersection, axis=1)

    # Aggregate by latitude and longitude
    result = df.groupby(["min_poi_lat", "min_poi_lon"], as_index=False)["intersection_time"].sum()
    
    # sort by intersection time
    result = result.sort_values(by="intersection_time", ascending=False)
    
    return result


In [5]:
# df_file_name = os.listdir(out_dir)[0]
def get_states_and_actions(df_file_name):
    try:
        df_path = os.path.join(out_dir, df_file_name)
        df = pd.read_csv(df_path)
        
        import copy
        df_copy = copy.deepcopy(df)
        
        night_stops = get_nighttime_stops(df_copy)
        most_visted_stop_lat = night_stops.iloc[0]["min_poi_lat"]
        most_visted_stop_lon = night_stops.iloc[0]["min_poi_lon"]
        home_lat_lon = (most_visted_stop_lat, most_visted_stop_lon)

        poi_embeddings_list = []
        for row_index in range(len(df)):
            df_poi_lat = df.iloc[row_index]["min_poi_lat"]
            df_poi_lon = df.iloc[row_index]["min_poi_lon"]
            poi_lat_lon = (df_poi_lat, df_poi_lon)
            if poi_lat_lon != home_lat_lon:
                df_poi_embedding = poi_embeddings_dict[(df_poi_lat, df_poi_lon)]
                # df.at[row_index, "poi_embedding"] = str(df_poi_embedding)
                poi_embeddings_list.append(df_poi_embedding)
            else:
                # print("Found home location")
                df_poi_embedding = [0]*len(poi_types_odered) + [1, 0]
                poi_embeddings_list.append(df_poi_embedding)
                
            
        # append rows with zeros to the poi_embeddings_list to make it with 221 rows
        poi_embeddings_list = poi_embeddings_list + [[0]*len(poi_types_odered) + [0, 1]]*(221-len(poi_embeddings_list))

        final_data = dict()
        final_data["obs"] = np.array(poi_embeddings_list)
        
        import datetime
        # type(df["stay_start_timestamp"].iloc[0])
        sp_time_duration_list = []
        min_day_of_year, max_day_of_year = 274, 305
        for row_index in range(len(df)):
            stay_start_timestamp = df.iloc[row_index]["stay_start_timestamp"]
            
            stay_start_timestamp = datetime.datetime.strptime(stay_start_timestamp, "%Y-%m-%d %H:%M:%S+00:00")
            
            # day of the year
            day_of_year = stay_start_timestamp.timetuple().tm_yday
            day_of_year_normalized = (day_of_year - min_day_of_year) / (1.0*(max_day_of_year - min_day_of_year))
            # hour of the day
            hour_of_day = stay_start_timestamp.hour / 24.0
            # minute of the hour
            minute_of_hour = stay_start_timestamp.minute / 60.0
            
            # print(day_of_year, hour_of_day, minute_of_hour)
            stay_duration_in_secs = df.iloc[row_index]["stay_duration_in_secs"]
            # convert stay_duration_in_secs from seconds to (num_days, num_hours, num_minutes, num_seconds)
            num_days = stay_duration_in_secs // (24*60*60)
            num_hours = (stay_duration_in_secs % (24*60*60)) // (60*60)
            num_minutes = (stay_duration_in_secs % (60*60)) // 60
            num_seconds = stay_duration_in_secs % 60
            
            normalized_num_days = num_days / (max_day_of_year - min_day_of_year)
            normalized_num_hours = num_hours / 24.0
            normalized_num_minutes = num_minutes / 60.0
            normalized_num_seconds = num_seconds / 60.0
            
            sp_time_duration_list.append([day_of_year_normalized, 
                                        hour_of_day, 
                                        minute_of_hour, 
                                        normalized_num_days, 
                                        normalized_num_hours, 
                                        normalized_num_minutes])
            
        # add more rows with zeros to make it 221 rows
        sp_time_duration_list = sp_time_duration_list + [[0]*6]*(221-len(sp_time_duration_list))
            
        final_data["act"] = sp_time_duration_list
        
        attention_mask = np.array([1]*len(df) + [0]*(221-len(df)))
        final_data["att_mask"] = attention_mask
        
        npzs_dir = f"/datassd1_8tb/users/chandrakanth/p2t4_exp_outs/sp_npzs_47class/{sim_name}__train/"
        if not os.path.exists(npzs_dir):
            os.makedirs(npzs_dir)
        # npz_file_name = df_file_name.replace(".csv", ".npz")
        agent_id = df_file_name.split("_")[1]
        npz_file_name = f"agent_{agent_id}.npz"
        npz_file_path = os.path.join(npzs_dir, npz_file_name)
        np.savez(npz_file_path, **final_data)
        
        return final_data
    except Exception as e:
        print(f"Error in processing {df_file_name}: {e}")
        import traceback
        traceback.print_exc()
        return None

In [ ]:
agents_1k_list = os.listdir(out_dir)[:1010]

In [7]:
from tqdm import tqdm
res_list = []
for agent_id_temp in tqdm(agents_1k_list):
    res = get_states_and_actions(agent_id_temp)
    res_list.append((res, agent_id_temp))
    # break

  4%|▍         | 45/1010 [00:00<00:11, 80.95it/s]Traceback (most recent call last):
  File "/tmp/ipykernel_2719348/9832419.py", line 5, in get_states_and_actions
    df = pd.read_csv(df_path)
         ^^^^^^^^^^^^^^^^^^^^
  File "/home/chandrakanth/ck_code/p1t2_gen_fas_3/.conda/lib/python3.11/site-packages/pandas/io/parsers/readers.py", line 1026, in read_csv
    return _read(filepath_or_buffer, kwds)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/chandrakanth/ck_code/p1t2_gen_fas_3/.conda/lib/python3.11/site-packages/pandas/io/parsers/readers.py", line 620, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/chandrakanth/ck_code/p1t2_gen_fas_3/.conda/lib/python3.11/site-packages/pandas/io/parsers/readers.py", line 1620, in __init__
    self._engine = self._make_engine(f, self.engine)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/chandrakanth/ck_code/p1t2_gen_fas_3/.conda/l

Error in processing agent_100738_sp_df.csv: No columns to parse from file


  8%|▊         | 82/1010 [00:00<00:11, 82.75it/s]Traceback (most recent call last):
  File "/tmp/ipykernel_2719348/9832419.py", line 5, in get_states_and_actions
    df = pd.read_csv(df_path)
         ^^^^^^^^^^^^^^^^^^^^
  File "/home/chandrakanth/ck_code/p1t2_gen_fas_3/.conda/lib/python3.11/site-packages/pandas/io/parsers/readers.py", line 1026, in read_csv
    return _read(filepath_or_buffer, kwds)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/chandrakanth/ck_code/p1t2_gen_fas_3/.conda/lib/python3.11/site-packages/pandas/io/parsers/readers.py", line 620, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/chandrakanth/ck_code/p1t2_gen_fas_3/.conda/lib/python3.11/site-packages/pandas/io/parsers/readers.py", line 1620, in __init__
    self._engine = self._make_engine(f, self.engine)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/chandrakanth/ck_code/p1t2_gen_fas_3/.conda/l

Error in processing agent_439056_sp_df.csv: No columns to parse from file


 14%|█▎        | 137/1010 [00:01<00:10, 81.81it/s]Traceback (most recent call last):
  File "/tmp/ipykernel_2719348/9832419.py", line 5, in get_states_and_actions
    df = pd.read_csv(df_path)
         ^^^^^^^^^^^^^^^^^^^^
  File "/home/chandrakanth/ck_code/p1t2_gen_fas_3/.conda/lib/python3.11/site-packages/pandas/io/parsers/readers.py", line 1026, in read_csv
    return _read(filepath_or_buffer, kwds)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/chandrakanth/ck_code/p1t2_gen_fas_3/.conda/lib/python3.11/site-packages/pandas/io/parsers/readers.py", line 620, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/chandrakanth/ck_code/p1t2_gen_fas_3/.conda/lib/python3.11/site-packages/pandas/io/parsers/readers.py", line 1620, in __init__
    self._engine = self._make_engine(f, self.engine)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/chandrakanth/ck_code/p1t2_gen_fas_3/.conda/

...

 69%|██████▊   | 694/1010 [00:08<00:03, 82.23it/s]Traceback (most recent call last):
  File "/tmp/ipykernel_2719348/9832419.py", line 5, in get_states_and_actions
    df = pd.read_csv(df_path)
         ^^^^^^^^^^^^^^^^^^^^
  File "/home/chandrakanth/ck_code/p1t2_gen_fas_3/.conda/lib/python3.11/site-packages/pandas/io/parsers/readers.py", line 1026, in read_csv
    return _read(filepath_or_buffer, kwds)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/chandrakanth/ck_code/p1t2_gen_fas_3/.conda/lib/python3.11/site-packages/pandas/io/parsers/readers.py", line 620, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/chandrakanth/ck_code/p1t2_gen_fas_3/.conda/lib/python3.11/site-packages/pandas/io/parsers/readers.py", line 1620, in __init__
    self._engine = self._make_engine(f, self.engine)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/chandrakanth/ck_code/p1t2_gen_fas_3/.conda/

Error in processing agent_95210_sp_df.csv: No columns to parse from file


 85%|████████▍ | 856/1010 [00:10<00:01, 82.83it/s]Traceback (most recent call last):
  File "/tmp/ipykernel_2719348/9832419.py", line 5, in get_states_and_actions
    df = pd.read_csv(df_path)
         ^^^^^^^^^^^^^^^^^^^^
  File "/home/chandrakanth/ck_code/p1t2_gen_fas_3/.conda/lib/python3.11/site-packages/pandas/io/parsers/readers.py", line 1026, in read_csv
    return _read(filepath_or_buffer, kwds)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/chandrakanth/ck_code/p1t2_gen_fas_3/.conda/lib/python3.11/site-packages/pandas/io/parsers/readers.py", line 620, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/chandrakanth/ck_code/p1t2_gen_fas_3/.conda/lib/python3.11/site-packages/pandas/io/parsers/readers.py", line 1620, in __init__
    self._engine = self._make_engine(f, self.engine)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/chandrakanth/ck_code/p1t2_gen_fas_3/.conda/

Error in processing agent_437510_sp_df.csv: No columns to parse from file
Error in processing agent_395974_sp_df.csv: No columns to parse from file


100%|██████████| 1010/1010 [00:12<00:00, 83.24it/s]


In [5]:
npzs_dir = f"/datassd1_8tb/users/chandrakanth/p2t4_exp_outs/sp_npzs_47class/{sim_name}__train/"
npz_file_name = (os.listdir(npzs_dir))[0]
npz_file_path = os.path.join(npzs_dir, npz_file_name)
npz_file_data = np.load(npz_file_path)

FileNotFoundError: [Errno 2] No such file or directory: '/datassd1_8tb/users/chandrakanth/p2t4_exp_outs/sp_npzs_47class/l3harris__train/'

In [14]:
npz_file_data["obs"]

array([[0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])